# Import basic packages (e.g., numpy and pandas)

In [429]:
import numpy as np
import pandas as pd
import datetime

# Read in and prepare the data set 

In [430]:
# read in the check-in file
data = pd.read_csv("/Users/KVTran/Documents/Research/WPI/Slip_Buddy/REU_2017/data/questions.csv")
# show the columns' names
list(data)
# select only a number of informative columns
col = ['DAY', 'patientid', 'date', 'Gender', 'Age', 'Marital Status', 'Education (Numeric)', 
       'Hispanic', 'Caucasian', 'Black', 'Asian', 'NHPI', 'Native American', 'Multi', 'Others',
       'Household', 'Income', 
       'AM Stress', 'AM Hunger', 'AM Sleep', 'AM Sleep hours',
       'AM Weight', 'Percent Weight change (from prev week)', 'Percent Weight change (from prev day)',
       'PM Stress','EVE Stress',
       'Number of Episodes Previous Day',
       'Episode']
data = data[col]
#data[0:10]

# Convert data to numerical

In [431]:
# Convert the categorical variables into numerical one
# Use the fit_transform() method from sklean.preprocessing.LableEncoder()
# Gender and Marital Status are catoegorical, not boolean. Hence, we add 1
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def toNumeric(data, nominalVariables):
    """
    This function takes in a list of categorical variables' names (columns' names) 
    It convert those values into numeric, and assign it directly to data dataframe
    Return nothing
    """
    for i in nominalVariables:
        numericalVals = le.fit_transform(data[i])
        if (i == "Gender" or i == "Marital Status"):
            numericalVals += 1
        data[i] = numericalVals
nominalVariables = ["Gender", "Marital Status","Hispanic", "Episode"]
toNumeric(data, nominalVariables)

# We remove the very last row that was added in when the data being converted to numerical
data = data.drop(data.index[-1])
#data[0:10]


# Find out which columns contain NaN

In [432]:
# Check for NaN values in every columns
def checkNaN(data, col):  
    """
    This function accesses column to column and checks if there is any NaN
    If yes, print out the column
    """
    nullCount = 0
    colWithNaN = []
    for i in col:
        if data[i].isnull().any():
            colWithNaN.append(i)
            nullCount += 1
    if nullCount == 0:
        print("no null values found")
    else:
        return colWithNaN    
colWithNaN = checkNaN(data, col)
#colWithNaN

# Adjust the Episode variable based on number of episode the day before

In [433]:
# The conflict between date format of Excel and Python was troublesome
# Issue solved by setting custom Excel format for date
# Choose the column -> format cell -> date -> choose m/d/yyyy -> choose custom -> mm/dd/yyy
def isConsecDate(date1, date2):
    """
    This function check on date1 and date2. Both are str() type
    If they are consecutive, return True
    """
    d = datetime.datetime.strptime(date1, '%m/%d/%Y') + datetime.timedelta(days=1)
    d = d.strftime('%m/%d/%Y')
    if (date2 == d):
        return True
    else:
        return False
    
def adjustEpisode(data):
    """
    
    """
    numRow = data.shape[0]
    for i in range(1, numRow):
        currentDate = data.date[i]
        prevDate = data.date[i-1]
        if isConsecDate(prevDate, currentDate):
            numEpPrevDay = data["Number of Episodes Previous Day"][i]
            if numEpPrevDay == 0:
                data.Episode.loc[i-1] = 0
            else:
                data.Episode.loc[i-1] = 1

adjustEpisode(data)
                


# Replace NaN with appropriate values

In [434]:
#
def replaceNaN(data, col):
    """
    Form a data block of rows as each patientid and each of the columns that contain NaN
    Compute the mean value of that data block
    replace the NaN value with that mean
    Return nothing
    """
    colWithNaN = checkNaN(data, col)
    patientID = list(set(data.patientid))
    for i in patientID:
        for j in colWithNaN:
            dataBlock = data[data.patientid == i][j]
            meanVal = np.mean(dataBlock)
            # replaceVal is a where() object of numpy.
            # check where the datablock contains NaN, replace that spot with meanVal.
            replaceVal = np.where(dataBlock.isnull(),
                                 -1,
                                 dataBlock)
            # Notice: using .loc[,] to assign value to the df itself instead of to a copy
            data.loc[data["patientid"] == i, j] = replaceVal

replaceNaN(data, col)

# Export the final data set

In [435]:
data.to_csv("/Users/KVTran/Documents/Research/WPI/Slip_Buddy/REU_2017/data/questions_cleaned.csv", index = False)